In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import os
import sys

import matplotlib.pyplot as plt
from matplotlib import colors

from ez_data import *
from pyPulses.utils import sweepMeasureParallelepiped, SweepMeasureParallelepipedConfig

If you wish to use GPIB instruments, make sure NI488.2 is installed.


In [16]:
# Create the Dataset Object
path = r'data/demo'
DS = Dataset(path, storage_backend = 'sqlite')

variable_attrs = dict(
    Vtg = {'long_name': r'$V_{tg}$', 'units': r'V'},
    Vbg = {'long_name': r'$V_{bg}$', 'units': r'V'},
    R   = {'long_name': r'R', 'units': r'$\Omega$'}
)

global_attrs = dict(
    B = {'long_name': r'B', 'units': r'T', 'value': 0.0}
)

DS.add_variable_attrs(variable_attrs)
DS.add_global_attrs(global_attrs)

In [3]:
# Set up the sweep
swept_names = ("Vtg", "Vbg")
measured_names = ("R",)

def post_callback(index, coords, data):
    coord_dict = dict(zip(swept_names, coords))
    data_dict = dict(zip(measured_names, data))
    DS.add_point(index, coord_dict, data_dict)

origin = [5.0, 5.0]
endpoints = np.array([[6.0, 6.0],
                      [6.0,  4.0]])
shape = [101, 11]

def dummy_set(val): pass
def dummy_get(): return np.random.exponential(scale=1e6)

para_config = SweepMeasureParallelepipedConfig(
    setters=(dummy_set, dummy_set),
    getters=(dummy_get,),
    swept_name=swept_names,
    measured_name=measured_names,
    time_per_point=0.01,
    origin=origin,
    endpoints=endpoints,
    shape=shape,
    post_callback=post_callback,
    retain_return=False
)

In [4]:
# Run the sweep
sweepMeasureParallelepiped(para_config)
# write metadata and flush any unsaved data
DS.write_metadata()
DS.flush()

In [17]:
# Load the data as a pandas dataframe

df = DS.load_dataframe()
df

,dim0,dim1,Vtg,Vbg,R
0,0,0,5.0,5.0,1.396437e+06
1,0,1,5.1,4.9,1.048724e+06
2,0,2,5.2,4.8,2.169487e+06
3,0,3,5.3,4.7,5.479093e+05
4,0,4,5.4,4.6,1.672194e+06
...,...,...,...,...,...
1106,100,6,6.6,5.4,7.417343e+05
1107,100,7,6.7,5.3,8.201682e+05
1108,100,8,6.8,5.2,1.112337e+06
1109,100,9,6.9,5.1,1.397400e+05


In [18]:
# Load the data as an xarray Dataset

ds = DS.to_xarray()
ds

<xarray.Dataset> Size: 27kB
Dimensions:  (dim0: 101, dim1: 11)
Coordinates:
    Vtg      (dim0, dim1) float64 9kB 5.0 5.1 5.2 5.3 5.4 ... 6.7 6.8 6.9 7.0
    Vbg      (dim0, dim1) float64 9kB 5.0 4.9 4.8 4.7 4.6 ... 5.3 5.2 5.1 5.0
Dimensions without coordinates: dim0, dim1
Data variables:
    R        (dim0, dim1) float64 9kB 1.396e+06 1.049e+06 ... 4.218e+04
Attributes:
    B:        {'long_name': 'B', 'units': 'T', 'value': 0.0}

In [24]:
# Plot the data in the xarray dataset as an example
%matplotlib qt5

ds["R"].plot.pcolormesh(x="Vtg", y ="Vbg", cmap = 'inferno', norm = colors.LogNorm(1e3, 1e6))
plt.show()

In [8]:
apply_linear_transform(df, np.array([[1, 1],[1, -1]]), ["Vtg", "Vbg"], ("n", "D"))

,dim0,dim1,Vtg,Vbg,R,n,D
0,0,0,5.0,5.0,1.396437e+06,10.0,0.0
1,0,1,5.1,4.9,1.048724e+06,10.0,0.2
2,0,2,5.2,4.8,2.169487e+06,10.0,0.4
3,0,3,5.3,4.7,5.479093e+05,10.0,0.6
4,0,4,5.4,4.6,1.672194e+06,10.0,0.8
...,...,...,...,...,...,...,...
1106,100,6,6.6,5.4,7.417343e+05,12.0,1.2
1107,100,7,6.7,5.3,8.201682e+05,12.0,1.4
1108,100,8,6.8,5.2,1.112337e+06,12.0,1.6
1109,100,9,6.9,5.1,1.397400e+05,12.0,1.8


In [22]:
ds = apply_linear_transform(ds, np.array([[1, 1],[1, -1]]), ["Vtg", "Vbg"], ("n", "D"))
ds

<xarray.Dataset> Size: 44kB
Dimensions:  (dim0: 101, dim1: 11)
Coordinates:
    Vtg      (dim0, dim1) float64 9kB 5.0 5.1 5.2 5.3 5.4 ... 6.7 6.8 6.9 7.0
    Vbg      (dim0, dim1) float64 9kB 5.0 4.9 4.8 4.7 4.6 ... 5.3 5.2 5.1 5.0
    n        (dim0, dim1) float64 9kB 10.0 10.0 10.0 10.0 ... 12.0 12.0 12.0
    D        (dim0, dim1) float64 9kB 0.0 0.2 0.4 0.6 0.8 ... 1.4 1.6 1.8 2.0
Dimensions without coordinates: dim0, dim1
Data variables:
    R        (dim0, dim1) float64 9kB 1.396e+06 1.049e+06 ... 4.218e+04
Attributes:
    B:        {'long_name': 'B', 'units': 'T', 'value': 0.0}

In [23]:
# Plot the data in the xarray dataset as an example
%matplotlib qt5

ds["R"].plot.pcolormesh(x="n", y ="D", cmap = 'inferno', norm = colors.LogNorm(1e3, 1e6))
plt.show()

In [11]:
def nD_from_V(Vtg, Vbg):
    return Vtg + Vbg, Vtg - Vbg

In [12]:
apply_transform(ds, nD_from_V, ["Vtg", "Vbg"], ("n", "D"))

<xarray.Dataset> Size: 44kB
Dimensions:  (dim0: 101, dim1: 11)
Coordinates:
    Vtg      (dim0, dim1) float64 9kB 5.0 5.1 5.2 5.3 5.4 ... 6.7 6.8 6.9 7.0
    Vbg      (dim0, dim1) float64 9kB 5.0 4.9 4.8 4.7 4.6 ... 5.3 5.2 5.1 5.0
    n        (dim0, dim1) float64 9kB 10.0 10.0 10.0 10.0 ... 12.0 12.0 12.0
    D        (dim0, dim1) float64 9kB 0.0 0.2 0.4 0.6 0.8 ... 1.4 1.6 1.8 2.0
Dimensions without coordinates: dim0, dim1
Data variables:
    R        (dim0, dim1) float64 9kB 1.396e+06 1.049e+06 ... 4.218e+04
Attributes:
    B:        {'long_name': 'B', 'units': 'T', 'value': 0.0}

In [13]:
apply_transform(df, nD_from_V, ["Vtg", "Vbg"], ("n", "D"))

,dim0,dim1,Vtg,Vbg,R,n,D
0,0,0,5.0,5.0,1.396437e+06,10.0,0.0
1,0,1,5.1,4.9,1.048724e+06,10.0,0.2
2,0,2,5.2,4.8,2.169487e+06,10.0,0.4
3,0,3,5.3,4.7,5.479093e+05,10.0,0.6
4,0,4,5.4,4.6,1.672194e+06,10.0,0.8
...,...,...,...,...,...,...,...
1106,100,6,6.6,5.4,7.417343e+05,12.0,1.2
1107,100,7,6.7,5.3,8.201682e+05,12.0,1.4
1108,100,8,6.8,5.2,1.112337e+06,12.0,1.6
1109,100,9,6.9,5.1,1.397400e+05,12.0,1.8


In [ ]:
# NOT RECOMMENDED

import mpld3

%matplotlib qt5

fig, ax = plt.subplots()
ds["R"].plot.pcolormesh(ax = ax, x="n", y ="D", cmap = 'inferno', norm = colors.LogNorm(1e3, 1e6))

html_str = mpld3.fig_to_html(fig)
with open("interactive_plot.html", "w") as Html_file:
    Html_file.write(html_str)

plt.show()

# Colorbar interactivity is broken. Prefer just saving static images.

c:\Users\avdif\Documents\ez_data\.venv\Lib\site-packages\mpld3\mplexporter\utils.py:300: MatplotlibDeprecationWarning: The converter attribute was deprecated in Matplotlib 3.10 and will be removed in 3.12. Use get_converter and set_converter methods instead.
  isinstance(axis.converter, matplotlib.dates._SwitchableDateConverter)
c:\Users\avdif\Documents\ez_data\.venv\Lib\site-packages\mpld3\mplexporter\utils.py:303: MatplotlibDeprecationWarning: The converter attribute was deprecated in Matplotlib 3.10 and will be removed in 3.12. Use get_converter and set_converter methods instead.
  isinstance(axis.converter, matplotlib.dates.DateConverter)
c:\Users\avdif\Documents\ez_data\.venv\Lib\site-packages\mpld3\mplexporter\utils.py:306: MatplotlibDeprecationWarning: The converter attribute was deprecated in Matplotlib 3.10 and will be removed in 3.12. Use get_converter and set_converter methods instead.
  isinstance(axis.converter, matplotlib.dates.ConciseDateConverter)
c:\Users\avdif\Documen

In [2]:
# Smarty Sweep / Special Measure Compatibility

sm_data_folder = r'C:\Users\avdif\MIT Dropbox\Andrew DiFabbio\Capacitance_2059\S91_2059He3\data'

def Vraw_to_V(Vtg, Vc):
    return Vtg - Vc, -Vc

q = 1.60217663e-19      # C
ε0 = 8.8541878188e-12   # F⋅m−1
Ct = 7.4397375e+15 * q  # F⋅m−2
Cb = 7.6688751e+15 * q  # F⋅m−2
n1 = 7.7093901e+15      # m−2
n0 = 2.9623501e+15      # m−2
D0 = -6730550           # V⋅m−1
def V_to_nD(Vtg, Vbg):
    n = (Ct*Vtg + Cb*Vbg)/q - n0
    D = (Ct*Vtg - Cb*Vbg)/(2*ε0) - D0
    return n, D

# This iterator pattern for the indices is common of smarty sweep /
# special measure data (often we split files by lines)
dim0 = 0
def idx_func(df):
    # Convert from raw voltages to effective Vtg and Vbg
    df = apply_transform(df, Vraw_to_V, ('Vtg', 'Vc'), ('Vtg', 'Vbg'))
    # Add n and D columns
    df = apply_transform(df, V_to_nD, ('Vtg', 'Vbg'), ('n', 'D'))
    # Add index
    global dim0
    idx0 = dim0
    dim0 += 1
    df = df.reset_index(names = 'dim1')
    df['dim0'] = idx0
    return df

col_names = {
    'timestamp' : '#Timestamp',
    'Xoff'      : 'X_{offbal} (V)',
    'Yoff'      : 'Y_{offbal} (V)',
    'Cex'       : 'C/C_{std}',
    'Closs'     : 'C_{loss}/C_{std}',
    'Vc'        : 'V_{Contacts} (V)',
    'Ic'        : 'I_{Contacts} (A)',
    'Vtg'       : 'V_{TG} (V)',
    'Itg'       : 'I_{TG} (A)',
    'f'         : '*Freq (Hz)',
    'B'         : '*B (T)',
    'Vexc'      : '*Excitation (V)'
}

sm_df = dat_to_pandas(sm_data_folder, "_scan149_map_340mK.dat", idx_func, col_names)
sm_df

c:\Users\avdif\Documents\ez_data\ez_data\smarty_sweep_utils.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  D = pd.concat([D, df.copy()], ignore_index=True)


,dim1,timestamp,Xoff,Yoff,Cex,Closs,Vc,Ic,Vtg,Itg,f,B,Vexc,Vbg,n,D,dim0
0,0,2025-06-05T08:19:41.970,-0.000026,-0.000025,1.01055,0.046859,5.70916,0,6.56504,0,7777780.0,0,0.015,-5.70916,2.096989e+15,8.447613e+08,0
1,1,2025-06-05T08:19:42.404,-0.000025,-0.000023,1.01054,0.045868,5.70886,0,6.56534,0,7777780.0,0,0.015,-5.70886,2.101522e+15,8.447606e+08,0
2,2,2025-06-05T08:19:42.828,NaN,NaN,NaN,NaN,5.70856,0,6.56564,0,7777780.0,0,0.015,-5.70856,2.106054e+15,8.447600e+08,0
3,3,2025-06-05T08:19:43.251,0.001753,-0.001143,2.01085,0.040909,5.70826,0,6.56594,0,7777780.0,0,0.015,-5.70826,2.110587e+15,8.447594e+08,0
4,4,2025-06-05T08:19:43.660,0.001753,-0.001143,2.01058,0.040734,5.70795,0,6.56625,0,7777780.0,0,0.015,-5.70795,2.115271e+15,8.447587e+08,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40035,996,2025-06-05T13:11:41.749,-0.000022,-0.000019,1.01056,0.043706,5.29927,0,6.76003,0,7777780.0,0,0.015,-5.29927,6.691059e+15,8.294463e+08,39
40036,997,2025-06-05T13:11:42.146,-0.000022,-0.000019,1.01051,0.043558,5.29897,0,6.76033,0,7777780.0,0,0.015,-5.29897,6.695591e+15,8.294456e+08,39
40037,998,2025-06-05T13:11:42.572,-0.000023,-0.000019,1.01041,0.043612,5.29867,0,6.76063,0,7777780.0,0,0.015,-5.29867,6.700124e+15,8.294450e+08,39
40038,999,2025-06-05T13:11:43.011,-0.000022,-0.000019,1.01062,0.043659,5.29837,0,6.76093,0,7777780.0,0,0.015,-5.29837,6.704657e+15,8.294444e+08,39


In [3]:
coord_attr = {
    'Vtg'   : {'long_name': r'$V_{tg}$', 'units': 'V'},
    'Vbg'   : {'long_name': r'$V_{bg}$', 'units': 'V'},
    'n'     : {'long_name': r'$n$', 'units': r'm$^{-2}$'},
    'D'     : {'long_name': r'$D$', 'units': 'V/m'},
    'B'     : {'long_name': r'$B$', 'units': 'T'}
}
data_attr = {
    'Xoff'  : {'long_name': r"X_{off}", 'units': 'V'},
    'Yoff'  : {'long_name': r"Y_{off}", 'units': 'V'},
    'Cex'   : {'long_name': r"$C_{ex}/C_{std}$"},
    'Closs' : {'long_name': r"$C_{loss}/C_{std}$"},
    'timestamp': {'long_name': 'time'}
}
global_attr = {
    'f'     : {'long_name': r'$f$', 'units': 'Hz', 'value': sm_df['f'].mean()},
    'Vexc'  : {'long_name': r'$V_{exc}$', 'units': 'V', 'value': sm_df['Vexc'].mean()}
}
# Unused for coord_attr and data_attr, but also not indices
ignored_attr = ['Ic', 'Itg', 'f', 'Vexc', 'Vc']

sm_ds = pandas_to_xarray(sm_df, coord_attr, data_attr, global_attr, ignored_attr)
sm_ds

<xarray.Dataset> Size: 3MB
Dimensions:    (dim1: 1001, dim0: 40)
Coordinates:
    Vtg        (dim1, dim0) float64 320kB 6.565 6.562 6.559 ... 6.764 6.761
    Vbg        (dim1, dim0) float64 320kB -5.709 -5.706 -5.704 ... -5.301 -5.298
    n          (dim1, dim0) float64 320kB 2.097e+15 2.097e+15 ... 6.709e+15
    D          (dim1, dim0) float64 320kB 8.448e+08 8.444e+08 ... 8.294e+08
    B          (dim1, dim0) float64 320kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Dimensions without coordinates: dim1, dim0
Data variables:
    Xoff       (dim1, dim0) float64 320kB -2.6e-05 1.433e-05 ... -2.315e-05
    Yoff       (dim1, dim0) float64 320kB -2.455e-05 2.21e-05 ... -1.914e-05
    Cex        (dim1, dim0) float64 320kB 1.011 1.015 1.015 ... 1.011 1.01 1.01
    Closs      (dim1, dim0) float64 320kB 0.04686 0.0178 ... 0.04395 0.04396
    timestamp  (dim1, dim0) datetime64[s] 320kB 2025-06-05T08:19:41 ... 2025-...
Attributes:
    f:        {'long_name': '$f$', 'units': 'Hz', 'value': np.float64(7777780...
    Vexc:     {'long_name': '$V_{exc}$', 'units': 'V', 'value': np.float64(0....

In [4]:
%matplotlib qt5
sm_ds['Cex'].plot.pcolormesh(x='n', y='D', vmin = 1.0, vmax = 1.05, cmap = 'inferno')
plt.show()

In [5]:
%matplotlib qt5
sm_ds['Cex'].plot.pcolormesh(x='Vtg', y='Vbg', vmin = 1.0, vmax = 1.05, cmap = 'inferno')
plt.show()